In [54]:
import pandas as pd
import gc


In [34]:
file_path = '../data/labeled_sunset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)
print("File loaded successfully. First 5 rows:")
print(df.head())

C:\Users\micha\AppData\Local\Temp\ipykernel_9880\3763364777.py:4: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


File loaded successfully. First 5 rows:
   Unnamed: 0              userid       username  \
0           3            22240612  AlArabiya_Eng   
1           4             6135622     dw_espanol   
2           5  848416437030985728   ChangshaCity   
3           8  984429894829592576   pulsoguayaco   
4           9  807095565028917248  linjianyangbe   

                                            acctdesc  \
0  The Arab world’s leading source of global news...   
1  Desde Alemania para América Latina. Todo lo qu...   
2  Changsha, the capital of central China’s Hunan...   
3  🌐✈Blog de aviación, viajes y economía para via...   
4  Nature heals. Birding in China: best photos+vi...   

                     location  following  followers  totaltweets  \
0                       Dubai         46     921780       324925   
1             Berlin, Germany        160    1266110       157669   
2  People's Republic of China        261      47826         3634   
3                        🇪🇨🇺🇸         

# Drop useless columns #

In [35]:
cols_to_drop = [
    'username', 'location', 'usercreatedts', 'tweetid', 'hashtags', 'coordinates',
    'favorite_count', 'is_retweet', 'original_tweet_id', 'original_tweet_userid', 'original_tweet_username',
    'in_reply_to_status_id', 'in_reply_to_user_id', 'in_reply_to_screen_name', 'is_quote_status',
    'quoted_status_id', 'quoted_status_userid','extractedts' ,'quoted_status_username', 'id'
]

df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)
gc.collect()
print("Specified columns dropped and memory cleaned up.")

for col in df.columns:
    print(col)

Specified columns dropped and memory cleaned up.
Unnamed: 0
userid
acctdesc
following
followers
totaltweets
tweetcreatedts
retweetcount
text
language
label


# Get unique users #

In [38]:
unique_user_ids = df['userid'].unique()
# Create a new DataFrame with the unique user IDs
proc_df = pd.DataFrame({'userid': unique_user_ids})

print("New DataFrame created with unique user IDs:")
print(proc_df.head())
print(f"Total unique users: {len(proc_df)}")



New DataFrame created with unique user IDs:
               userid
0            22240612
1             6135622
2  848416437030985728
3  984429894829592576
4  807095565028917248
Total unique users: 104288


# Total tweets is max of the total tweets for each user #




In [40]:
# Group the original DataFrame by 'userid' and find the maximum 'totaltweets' for each user
max_total_tweets = df.groupby('userid')['totaltweets'].max().reset_index()

# Merge the results into the 'proc_df'
proc_df = pd.merge(proc_df, max_total_tweets, on='userid', how='left')

print("Updated 'proc_df' with 'totaltweets':")
print(proc_df.head())

Updated 'proc_df' with 'totaltweets':
               userid  totaltweets
0            22240612       347147
1             6135622       172270
2  848416437030985728         3634
3  984429894829592576        16116
4  807095565028917248         8325


# Following and Followers from the latest tweet #
* followers_following_ratio = following/followers
* if followers_following_ratio is not defined it will be the number of following

In [41]:
# Sort df by userid and tweetcreatedts (latest first)
df_sorted = df.sort_values(by=['userid', 'tweetcreatedts'], ascending=[True, False])

# Get the latest tweet for each user (first row after sorting)
latest_tweets = df_sorted.groupby('userid').first().reset_index()

# Select relevant columns
latest_user_info = latest_tweets[['userid', 'following', 'followers']]

# Calculate the followers_following_ratio, handling potential division by zero
latest_user_info['followers_following_ratio'] = latest_user_info.apply(
    lambda row: row['following'] / row['followers'] if row['followers'] != 0 else row['following'], axis=1
)


# Merge with proc_df
proc_df = pd.merge(proc_df, latest_user_info, on='userid', how='left')

print("Updated 'proc_df' with latest tweet info, following, followers, and ratio:")
print(proc_df.head())


Updated 'proc_df' with latest tweet info, following, followers, and ratio:
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  
0                   0.000050  
1                   0.000124  
2                   0.005457  
3                   0.219512  
4                   0.614297  


C:\Users\micha\AppData\Local\Temp\ipykernel_9880\1647166422.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_user_info['followers_following_ratio'] = latest_user_info.apply(


# avg_retweetcount #

In [43]:
# Group the original DataFrame by 'userid' and calculate the average 'retweetcount' for each user
temp_df = df.groupby('userid')['retweetcount'].mean().reset_index()

# Rename the column to be more descriptive before merging (optional but good practice)
temp_df.rename(columns={'retweetcount': 'avg_retweetcount'}, inplace=True)

# Merge the results into the 'proc_df'
proc_df = pd.merge(proc_df, temp_df, on='userid', how='left')

print("Updated 'proc_df' with 'avg_retweetcount':")
print(proc_df.head())

Updated 'proc_df' with 'avg_retweetcount':
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  
0                   0.000050          1.755378  
1                   0.000124         39.639711  
2                   0.005457          1.000000  
3                   0.219512          0.398857  
4                   0.614297         18.340000  


# Avg_words_per_tweet #

In [44]:
# Calculate word count for each tweet
# We'll handle potential NaN values in the 'text' column
df['word_count'] = df['text'].str.split().str.len().fillna(0)

# Group by 'userid' and calculate the average word count
temp_df = df.groupby('userid')['word_count'].mean().reset_index()

# Rename the column for clarity
temp_df.rename(columns={'word_count': 'avg_words_per_tweet'}, inplace=True)

# Merge into proc_df
proc_df = pd.merge(proc_df, temp_df, on='userid', how='left')

print("Updated 'proc_df' with 'avg_words_per_tweet':")
print(proc_df.head())

# delete word_count from df
if 'word_count' in df.columns:
    del df['word_count']
    print("Deleted 'word_count' from df")

Updated 'proc_df' with 'avg_words_per_tweet':
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  avg_words_per_tweet  
0                   0.000050          1.755378            30.008004  
1                   0.000124         39.639711            35.051940  
2                   0.005457          1.000000            45.000000  
3                   0.219512          0.398857            29.329143  
4                   0.614297         18.340000            24.560000  
Deleted 'word_count' from df


# daily_tweet_count #

In [46]:
# Ensure 'tweetcreatedts' is in datetime format
df['tweetcreatedts'] = pd.to_datetime(df['tweetcreatedts'])

# Extract the date from 'tweetcreatedts'
df['tweet_date'] = df['tweetcreatedts'].dt.date

# Group by userid and date, then count tweets per day
temp_df = df.groupby(['userid', 'tweet_date']).size().reset_index(name='daily_tweet_count')

# Group by userid and calculate the average daily tweet count
tweet_frequency_per_day = temp_df.groupby('userid')['daily_tweet_count'].mean().reset_index()

# Merge with proc_df
proc_df = pd.merge(proc_df, tweet_frequency_per_day, on='userid', how='left')

print("Updated 'proc_df' with 'tweet_frequency_per_day':")
print(proc_df.head())

if 'tweet_frequency_per_day' in globals():
    del tweet_frequency_per_day
    print("Deleted 'tweet_frequency_per_day'")

if 'tweet_date' in df.columns:
    del df['tweet_date']
    print("Deleted 'tweet_date' from df")

Updated 'proc_df' with 'tweet_frequency_per_day':
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  avg_words_per_tweet  \
0                   0.000050          1.755378            30.008004   
1                   0.000124         39.639711            35.051940   
2                   0.005457          1.000000            45.000000   
3                   0.219512          0.398857            29.329143   
4                   0.614297         18.340000            24.560000   

   daily_tweet_count  
0          14.077465  
1           5.318182  
2           1.000000  
3           4.729730  
4           1.785714  
Deleted 'tweet_frequency

# unique_language_count #

In [48]:
# Group by userid and language and count the occurrences
temp_df = df.groupby(['userid', 'language']).size().reset_index(name='language_count')

# Group by userid and count the number of unique languages
unique_language_count = temp_df.groupby('userid').size().reset_index(name='unique_language_count')

# Merge with proc_df
proc_df = pd.merge(proc_df, unique_language_count, on='userid', how='left')

print("Updated 'proc_df' with 'unique_language_count':")
print(proc_df.head())


if 'unique_language_count' in globals():
    del unique_language_count
    print("Deleted 'unique_language_count'")

Updated 'proc_df' with 'unique_language_count':
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  avg_words_per_tweet  \
0                   0.000050          1.755378            30.008004   
1                   0.000124         39.639711            35.051940   
2                   0.005457          1.000000            45.000000   
3                   0.219512          0.398857            29.329143   
4                   0.614297         18.340000            24.560000   

   daily_tweet_count  unique_language_count  
0          14.077465                      2  
1           5.318182                      2  
2           1.000000      

# max_tweers_per_hour #

In [58]:
# Ensure 'tweetcreatedts' is in datetime format
df['tweetcreatedts'] = pd.to_datetime(df['tweetcreatedts'])
# Sort df by userid and tweetcreatedts
df_sorted = df.sort_values(by=['userid', 'tweetcreatedts'])

# Count tweets per hour window per user
burstiness = (
    df_sorted
    .groupby('userid')
    .apply(lambda df: df.set_index('tweetcreatedts')
                      .rolling('1h')['userid']
                      .count()
                      .max())
    .reset_index(name='Max_tweets_in_1hr')
)


# Merge back
proc_df = pd.merge(proc_df, burstiness, on='userid', how='left')

print("Updated 'proc_df' with 'Max_tweets_in_1hr':")
print(proc_df.head())


Updated 'proc_df' with 'Max_tweets_in_1hr':
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  avg_words_per_tweet  \
0                   0.000050          1.755378            30.008004   
1                   0.000124         39.639711            35.051940   
2                   0.005457          1.000000            45.000000   
3                   0.219512          0.398857            29.329143   
4                   0.614297         18.340000            24.560000   

   daily_tweet_count  unique_language_count  Max_tweets_in_1hr  
0          14.077465                      2                7.0  
1           5.318182                  

C:\Users\micha\AppData\Local\Temp\ipykernel_9880\4035292727.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df.set_index('tweetcreatedts')


# Add label #

In [62]:
# Group by userid and get the first occurrence of the label for each user
temp_df = df.groupby('userid')['label'].first().reset_index(name='label')

# Map labels: "human" to 0, "bot" to 1
label_mapping = {"human": 0, "bot": 1}
temp_df['label'] = temp_df['label'].map(label_mapping)

# Merge the results into the 'proc_df'
proc_df = pd.merge(proc_df, temp_df, on='userid', how='left')

print("Updated 'proc_df' with 'label' (first instance and mapped):")
print(proc_df.head())


Updated 'proc_df' with 'label' (first instance and mapped):
               userid  totaltweets  following  followers  \
0            22240612       347147         46     925487   
1             6135622       172270        169    1367996   
2  848416437030985728         3634        261      47826   
3  984429894829592576        16116         72        328   
4  807095565028917248         8325      15984      26020   

   followers_following_ratio  avg_retweetcount  avg_words_per_tweet  \
0                   0.000050          1.755378            30.008004   
1                   0.000124         39.639711            35.051940   
2                   0.005457          1.000000            45.000000   
3                   0.219512          0.398857            29.329143   
4                   0.614297         18.340000            24.560000   

   daily_tweet_count  unique_language_count  Max_tweets_in_1hr  label  
0          14.077465                      2                7.0      0  
1       

# Memory cleanup #


In [65]:
"""Deletes temporary variables to free up memory."""
if 'unique_user_ids' in globals():
  del unique_user_ids
  print("Deleted 'unique_user_ids'")

if 'max_total_tweets' in globals():
  del max_total_tweets
  print("Deleted 'max_total_tweets'")

if 'df_sorted' in globals():
  del df_sorted
  print("Deleted 'df_sorted'")

if 'temp_df' in globals():
  del temp_df
  print("Deleted 'temp_df'")

if 'latest_user_info' in globals():
  del latest_user_info
  print("Deleted 'latest_user_info'")

Deleted 'temp_df'


# Make userdesc_labeled for the LM model #

In [ ]:
file_path = '../data/labeled_sunset.csv'
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Map string labels to numeric
df = df.drop_duplicates(subset=['id'], keep='first').copy()
df_clean = df[["userid", "acctdesc", "label"]].dropna()
label_map = {"human": 0, "bot": 1}
df_clean["label"] = df_clean["label"].map(label_map)

# Check result
print(df_clean["label"].value_counts())

# Drop rows with missing descriptions
df_clean = df_clean.dropna(subset=["acctdesc"]).copy()

# Optional: convert labels to int (in case they're strings)
df_clean['label'] = df_clean['label'].astype(int)

# Check balance
print(df_clean['label'].value_counts())
print(df_clean.head())

output_file_path = '../data/userdesc_labeled.csv'
df_clean.to_csv(output_file_path, index=False)

# Add userdesc to intersection for the ensemble model #

In [ ]:
# Load the labeled_intersection.csv file
intersection_file_path = '../data/labeled_intersection.csv'
df_intersection = pd.read_csv(intersection_file_path)

print("labeled_intersection.csv loaded successfully. First 5 rows:")
print(df_intersection.head())

# Merge df_intersection with df_clean to add the 'acctdesc'
# We use a left merge to keep all users from df_intersection
df_merged = pd.merge(df_intersection, df_clean[['userid', 'acctdesc']], on='userid', how='right')

print("\nMerged DataFrame with 'acctdesc'. First 5 rows:")
print(df_merged.head())

# For users in labeled_intersection.csv that were not in df_clean, the 'acctdesc' will be NaN (null)

# Save to a CSV files #

In [64]:
proc_df.to_csv('../data/processed_users.csv', index=False)
print("proc_df saved to '../data/processed_users.csv'")

proc_df saved to 'data/processed_users.csv'


In [7]:
df_intersection.to_csv('../data/labeled_intersection.csv', index=False)
print("df_intersection saved to '../data/labeled_intersection.csv'")

df_intersection saved to 'data/labeled_intersection.csv'


In [ ]:
output_file_path = '../data/intersection_userdesc_labeled.csv'
df_merged.to_csv(output_file_path, index=False)
